In [0]:
# Helper Function for logging

from pyspark.sql.functions import lit, current_timestamp
import uuid

def write_log(table_name, source_system, log_table):
    spark.createDataFrame(
        [(str(uuid.uuid4()), table_name, source_system)],
        ["log_id", "table_name", "source_system"]
    ) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("created_at", current_timestamp()) \
    .write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(log_table)

### Views for normalization

In [0]:
%sql
CREATE OR REPLACE VIEW capstone.default.v_silver_prices_usd AS
SELECT
    dp.trade_date,
    dp.ticker,
    dp.open_price * fx.fx_rate AS open_price_usd,
    dp.high_price * fx.fx_rate AS high_price_usd,
    dp.low_price * fx.fx_rate AS low_price_usd,
    dp.close_price * fx.fx_rate AS close_price_usd,
    dp.adjusted_close * fx.fx_rate AS adjusted_close_usd,
    dp.volume,
    c.sector,
    c.industry,
    c.currency

FROM capstone.default.silver_daily_prices dp
JOIN capstone.default.silver_companies c
  ON dp.ticker = c.ticker
JOIN capstone.default.silver_fx_rates fx
  ON dp.trade_date = fx.fx_date
 AND c.currency = fx.from_currency
 AND fx.to_currency = 'USD';

In [0]:
%sql
CREATE OR REPLACE VIEW capstone.default.v_silver_trades_usd AS
SELECT
    t.trade_date,
    t.trader_id,
    t.side,
    t.quantity,
    t.trade_value * fx.fx_rate AS trade_value_usd
FROM capstone.default.silver_trades t
JOIN capstone.default.silver_traders tr
  ON t.trader_id = tr.trader_id
JOIN capstone.default.silver_fx_rates fx
  ON t.trade_date = fx.fx_date
 AND tr.base_currency = fx.from_currency
 AND fx.to_currency = 'USD';


### Daily Market Data

In [0]:
%sql
-- Gold table for market daily KPIs
CREATE TABLE IF NOT EXISTS capstone.default.gold_market_daily_kpis_usd (
    trade_date DATE,

    total_volume BIGINT,
    avg_close_price_usd DOUBLE,
    market_daily_return DOUBLE,
    market_volatility_30d DOUBLE,
    active_companies INT,

    gold_processed_timestamp TIMESTAMP
)
USING DELTA;

In [0]:
%sql
WITH market_returns AS (
    SELECT
        trade_date,
        SUM(volume) AS total_volume,
        AVG(close_price_usd) AS avg_close_price_usd,

        (AVG(close_price_usd) -
         LAG(AVG(close_price_usd)) OVER (ORDER BY trade_date))
        / LAG(AVG(close_price_usd)) OVER (ORDER BY trade_date)
        AS market_daily_return,

        COUNT(DISTINCT ticker) AS active_companies
    FROM capstone.default.v_silver_prices_usd
    GROUP BY trade_date
)

INSERT OVERWRITE TABLE capstone.default.gold_market_daily_kpis_usd
SELECT
    trade_date,
    total_volume,
    avg_close_price_usd,
    market_daily_return,
    STDDEV(market_daily_return)
        OVER (
            ORDER BY trade_date
            ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
        ) AS market_volatility_30d,
    active_companies,
    current_timestamp() AS gold_processed_timestamp
FROM market_returns;


num_affected_rows,num_inserted_rows
1311,1311


In [0]:
write_log("capstone.default.daily_prices", "ingest", "capstone.default.gold_log")

### Company specific data

In [0]:
%sql
-- Gold table for company performance
CREATE TABLE IF NOT EXISTS capstone.default.gold_company_performance_usd (
    trade_date DATE,
    ticker STRING,
    close_price_usd DOUBLE,
    daily_return DOUBLE,
    moving_avg_20 DOUBLE,
    moving_avg_50 DOUBLE,
    volatility_30d DOUBLE,
    total_volume BIGINT,
    sector STRING,
    industry STRING,
    gold_processed_timestamp TIMESTAMP
)
USING DELTA;


In [0]:
%sql
INSERT OVERWRITE TABLE capstone.default.gold_company_performance_usd
SELECT
    trade_date,
    ticker,
    close_price_usd,
    daily_return,
    AVG(close_price_usd)
        OVER (PARTITION BY ticker ORDER BY trade_date
              ROWS BETWEEN 19 PRECEDING AND CURRENT ROW) AS moving_avg_20,
    AVG(close_price_usd)
        OVER (PARTITION BY ticker ORDER BY trade_date
              ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) AS moving_avg_50,
    STDDEV(daily_return)
        OVER (PARTITION BY ticker ORDER BY trade_date
              ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS volatility_30d,

    volume AS total_volume,
    sector,
    industry,
    current_timestamp() AS gold_processed_timestamp
FROM (
    SELECT
        trade_date,
        ticker,
        close_price_usd,
        volume,
        sector,
        industry,

        (close_price_usd -
         LAG(close_price_usd)
            OVER (PARTITION BY ticker ORDER BY trade_date))
        / LAG(close_price_usd)
            OVER (PARTITION BY ticker ORDER BY trade_date)
        AS daily_return
    FROM capstone.default.v_silver_prices_usd
);


num_affected_rows,num_inserted_rows
9177,9177


In [0]:
write_log("capstone.default.companies", "ingest", "capstone.default.gold_log")
write_log("capstone.default.fx_rates", "ingest", "capstone.default.gold_log")
write_log("capstone.default.daily_prices", "ingest", "capstone.default.gold_log")
write_log("capstone.default.traders", "ingest", "capstone.default.gold_log")

### Trader Portfolio

In [0]:
%sql
-- Gold table for trader portfolio performance
CREATE TABLE IF NOT EXISTS capstone.default.gold_trader_portfolio_performance_usd (
    trade_date DATE,
    trader_id STRING,

    total_buy_value_usd DOUBLE,
    total_sell_value_usd DOUBLE,

    daily_pnl_usd DOUBLE,
    cumulative_pnl_usd DOUBLE,

    total_trades INT,

    gold_processed_timestamp TIMESTAMP
)
USING DELTA;

In [0]:
 %sql
 INSERT OVERWRITE TABLE capstone.default.gold_trader_portfolio_performance_usd
 SELECT
     trade_date,
     trader_id,

     SUM(CASE WHEN side = 'BUY'
              THEN trade_value_usd ELSE 0 END) AS total_buy_value_usd,

     SUM(CASE WHEN side = 'SELL'
              THEN trade_value_usd ELSE 0 END) AS total_sell_value_usd,

     SUM(CASE WHEN side = 'SELL'
              THEN trade_value_usd ELSE -trade_value_usd END)
     AS daily_pnl_usd,

     SUM(SUM(CASE WHEN side = 'SELL'
                  THEN trade_value_usd ELSE -trade_value_usd END))
         OVER (PARTITION BY trader_id ORDER BY trade_date)
     AS cumulative_pnl_usd,

     COUNT(*) AS total_trades,
     current_timestamp() AS gold_processed_timestamp

 FROM capstone.default.v_silver_trades_usd
 GROUP BY trade_date, trader_id;

num_affected_rows,num_inserted_rows
17319,17319


In [0]:
write_log("capstone.default.trades", "ingest", "capstone.default.gold_log")

### Sector wise data

In [0]:
%sql
-- Gold table for sector performance
CREATE TABLE IF NOT EXISTS capstone.default.gold_sector_performance_usd (
    trade_date DATE,
    sector STRING,

    avg_sector_price_usd DOUBLE,
    sector_daily_return DOUBLE,
    sector_volatility_30d DOUBLE,

    total_volume BIGINT,

    gold_processed_timestamp TIMESTAMP
)
USING DELTA;


In [0]:
%sql
WITH sector_returns AS (
    SELECT
        trade_date,
        sector,
        AVG(close_price_usd) AS avg_sector_price_usd,

        (AVG(close_price_usd) -
         LAG(AVG(close_price_usd))
            OVER (PARTITION BY sector ORDER BY trade_date))
        / LAG(AVG(close_price_usd))
            OVER (PARTITION BY sector ORDER BY trade_date)
        AS sector_daily_return,

        SUM(volume) AS total_volume
    FROM capstone.default.v_silver_prices_usd
    GROUP BY trade_date, sector
)
INSERT OVERWRITE TABLE capstone.default.gold_sector_performance_usd
SELECT
    trade_date,
    sector,
    avg_sector_price_usd,
    sector_daily_return,

    STDDEV(sector_daily_return)
        OVER (PARTITION BY sector ORDER BY trade_date
              ROWS BETWEEN 29 PRECEDING AND CURRENT ROW)
        AS sector_volatility_30d,

    total_volume,
    current_timestamp() AS gold_processed_timestamp
FROM sector_returns;


num_affected_rows,num_inserted_rows
6555,6555


In [0]:
write_log("capstone.default.trades", "ingest", "capstone.default.gold_log")

### Monthly trends

In [0]:
%sql
CREATE TABLE IF NOT EXISTS capstone.default.gold_monthly_market_trends (
    year INT,
    month INT,
    total_volume BIGINT,
    avg_monthly_price_usd DOUBLE,
    monthly_return DOUBLE,
    rolling_3m_avg_price DOUBLE,
    gold_processed_timestamp TIMESTAMP
)
USING DELTA;

In [0]:
%sql
WITH monthly_returns AS (
    SELECT
        YEAR(trade_date) AS year,
        MONTH(trade_date) AS month,

        SUM(volume) AS total_volume,
        AVG(close_price_usd) AS avg_monthly_price_usd,

        (AVG(close_price_usd) -
         LAG(AVG(close_price_usd))
            OVER (ORDER BY YEAR(trade_date), MONTH(trade_date)))
        / LAG(AVG(close_price_usd))
            OVER (ORDER BY YEAR(trade_date), MONTH(trade_date))
        AS monthly_return
    FROM capstone.default.v_silver_prices_usd
    GROUP BY YEAR(trade_date), MONTH(trade_date)
)

INSERT OVERWRITE TABLE capstone.default.gold_monthly_market_trends
SELECT
    year,
    month,
    total_volume,
    avg_monthly_price_usd,
    monthly_return,

    AVG(avg_monthly_price_usd)
        OVER (ORDER BY year, month
              ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
        AS rolling_3m_avg_price,

    STDDEV(monthly_return)
        OVER (ORDER BY year, month
              ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
        AS monthly_volatility_3m
FROM monthly_returns;

num_affected_rows,num_inserted_rows
62,62


In [0]:
write_log("capstone.default.traders", "ingest", "capstone.default.gold_log")
write_log("capstone.default.companies", "ingest", "capstone.default.gold_log")
write_log("capstone.default.daily_prices", "ingest", "capstone.default.gold_log")
write_log("capstone.default.fx_rates", "ingest", "capstone.default.gold_log")